In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#construct dataset  
dataset구성
spam dataset을 하나씩 반환하며  
BertTokenzier를 통해 tokenziation후 input_ids, token_type_ids, attention mask를 tensor로 바꿔서 return  
또한 label이 spam일때는 1 ham일때는 0을 함께 return  

In [ ]:
import torch
import pandas as pd

from transformers import AutoTokenizer,BertTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader


data_path = "/content/drive/MyDrive/여름 NLP/6일차/dataset/spam.csv"
data_df = pd.read_csv(data_path,encoding="ISO-8859-1")

print(data_df)

data_df = data_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis= 1)

print(data_df.columns)

#index 0부터 시작하기위해 사용
train_df = data_df.loc[:4000,:].reset_index()
test_df = data_df.loc[4000:,:].reset_index()


#토큰화
tokenizer = get_tokenizer("basic_english")

ex_text = train_df.loc[0,"v2"]

#토큰화한걸 숫자로 변경
vocab = build_vocab_from_iterator(list(map(tokenizer,data_df.loc[:,"v2"])))

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class MyDataset(Dataset):
  def __init__(self,df,tokenizer) -> None:
      super().__init__()
      self.df = df
      self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
  
  def __len__(self):
    return len(self.df)
  
  def __getitem__(self,index):
    data = self.df.loc[index,"v2"]
    data = self.tokenizer(data, max_length=512,padding='max_length')
    
    if self.df.loc[index,"v1"]=="spam":
      label = 1
    elif self.df.loc[index,"v1"]=="ham":
      label = 0

    input_ids = torch.IntTensor(data["input_ids"])
    token_type_ids = torch.IntTensor(data["token_type_ids"])
    attention_mask = torch.IntTensor(data["attention_mask"])

    return input_ids, token_type_ids, attention_mask,label

train_dataset = MyDataset(train_df, tokenizer)
test_dataset = MyDataset(test_df, tokenizer)


batch_size = 3
train_dataloader = DataLoader(train_dataset,batch_size = batch_size)
test_dataloader = DataLoader(test_dataset,batch_size = batch_size)

for i in train_dataloader:
  input_ids = i[0]
  token_type_ids = i[1]
  attention_mask = i[2]
  label = i[3]
  print(input_ids.shape)
  print(token_type_ids.shape)
  print(attention_mask.shape)
  print(label)
  print(i)
  break








        v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN  
1           NaN        NaN  


##Model
BertModel을 이용해서 구현  
BertModel의 cls token에 대한 output인 pooler_output에 linear를 통해 2차원 벡터로 변환  
BertModel: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel

In [ ]:
from torch import nn
from transformers import BertModel


class MyModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.bert = BertModel.from_pretrained("bert-base-uncased")
      self.ln1 = nn.Linear(768,1)

  def forward(self,input_ids, token_type_ids, attention_mask, label):
    out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    out = out["last_hidden_state"]
    print(out.shape)
    out = self.ln1(out)
    print(out.shape)
    print(label.shape)


    return out.squeeze()
model = MyModel()

for i in train_dataloader:
  input_ids = i[0]
  token_type_ids = i[1]
  attention_mask = i[2]
  label = i[3]
  out = model(input_ids, token_type_ids, attention_mask,label )
  #print(out[0].size(),out[1].size())
  #print(i[3])
  break

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3])


##train
cross entropy loss와 Adam을 통해 train

In [ ]:
from torch.optim import Adam

model = MyModel()
model.cuda()

optimizer = Adam(model.parameters(), lr=0.0001) 
lf = nn.CrossEntropyLoss()

for e in range(30):
  print("\n\n epoch", e)
  model.train()
  train_ac = 0
  epoch_loss = 0

  for i in train_dataloader:
    optimizer.zero_grad()

    input_ids = i[0]
    input_ids = input_ids.cuda()
    token_type_ids = i[1]
    token_type_ids = token_type_ids.cuda()
    attention_mask = i[2]
    attention_mask = attention_mask.cuda()
    target = i[3]
    target = target.cuda()
    print(target.shape)

    output = model(input_ids,token_type_ids,attention_mask,target)
    print(output.shape)

    pred_label = torch.argmax(output, dim=-1)
    target = target.reshape(-1)

    train_acc = sum(pred_label == target.reshape(-1))
    train_ac += train_acc
    
    loss = lf(output,target)
    

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  print(train_acc)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_ac/len(train_dataset))

  model.eval()
  test_loss = 0
  test_ac = 0 

  with torch.no_grad():
    for i in test_dataloader:
      input_ids = i[0]
      input_ids = input_ids.cuda()
      token_type_ids = i[1]
      token_type_ids = token_type_ids.cuda()
      attention_mask = i[2]
      attention_mask = attention_mask.cuda()
      target = i[3]
      target = target.cuda()

      output = model(input_ids,token_type_ids,attention_mask)

      test_pred_label = torch.argmax(output, dim=-1)
      target = target.reshape(-1)

      test_acc = sum(test_pred_label == target.reshape(-1))
      test_ac += test_acc
    
      loss = lf(output,target)

      test_loss += loss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test acc", train_ac/len(test_dataset))


    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).




 epoch 0
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 512, 768])
torch.Size([3, 512, 1])
torch.Size([3, 512])
torch.Size([3])
torch.Size([3, 5

KeyboardInterrupt: ignored